In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same',
                    input_shape=(Chans, Samples, 1),
                    use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_regularizer=l2(0.001),
                             padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16),
                             use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    flatten = Flatten()(block2)

    dense = Dense(nb_classes, activation='softmax',
                  kernel_regularizer=l2(0.001))(flatten)

    model = Model(inputs=input1, outputs=dense)
    return model


In [2]:
import pandas as pd

# Load data
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")

# Extract EEG channels
eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X = []
y = []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values
    if data.shape[0] < 1000:  # pad if needed
        data = np.pad(data, ((0, 1000 - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:1000]  # crop
    X.append(data)
    label = group_sorted['label'].iloc[0]
    y.append(label_map[label])

X = np.array(X)  # shape: (n_trials, time_steps, 22)
y = to_categorical(y, num_classes=4)

# Reshape for EEGNet: (n_samples, channels, time, 1)
X = np.transpose(X, (0, 2, 1))  # (samples, channels, time)
X = X[..., np.newaxis]          # (samples, channels, time, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y.argmax(1), random_state=42)


In [3]:
model = EEGNet(nb_classes=4, Chans=22, Samples=1000)
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=200,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")


c:\Python\Python365\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.2618 - loss: 1.6313 - val_accuracy: 0.2347 - val_loss: 1.3959
Epoch 2/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.2640 - loss: 1.4912 - val_accuracy: 0.2857 - val_loss: 1.3938
Epoch 3/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.2758 - loss: 1.4583 - val_accuracy: 0.2857 - val_loss: 1.3931
Epoch 4/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.2452 - loss: 1.4568 - val_accuracy: 0.2857 - val_loss: 1.3931
Epoch 5/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.2840 - loss: 1.4237 - val_accuracy: 0.3138 - val_loss: 1.3927
Epoch 6/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.2488 - loss: 1.4222 - val_accuracy: 0.3036 - val_loss: 1.3902
Epoch 7/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.2722 - loss: 1.3969 - val_accuracy: 0.2832 - val_loss: 1.3828
Epoch 8/200
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - accuracy: 0.2654 - loss: 1.4121 - val_acc